In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import numpy as np
from plots_file import *
from our_functionsv3 import *

np.random.seed(0)
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

C:\Users\Pauli\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [39]:
X_train_1, Y_train_1 = get_data(pos = "train_pos.txt", neg = "train_neg.txt")

In [43]:
len(X_train_1)

200000

In [41]:
max_length = len(max(X_train_1, key=len).split())
print(max_length)

60


In [14]:
X_train, Y_train = get_data(pos = "train_pos_preprocessed.txt", neg = "train_neg_preprocessed.txt")

In [44]:
len(X_train)

196672

In [45]:
#most deep learning frameworks require that all sequences in the same mini-batch have the same length --> padding
max_length = len(max(X_train, key=len).split())
print(max_length)   #normal ????

1490


In [29]:
X_train = np.asarray(X_train)

In [30]:
Y_train_oh = convert_to_one_hot(Y_train, C=2)

In [31]:
ids, X_test = get_test_data("test_data.txt")
X_test = np.asarray(X_test)

In [11]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('./twitter/glove.twitter.27B.50d.txt')

### Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences

In [37]:
def sentences_to_indices(X, word_to_index, max_length):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """    
    m = len(X)                                   # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_length))
    
    for i in range(m):                               # loop over training examples        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [x.lower() for x in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            if w in word_to_index.keys():
            # Set the (i,j)th entry of X_indices to the index of the correct word.
                X_indices[i, j] = word_to_index[w]
            else:
                X_indices[i, j] = -1   #si il ne connait pas le mot il met -1, voir comment on gère ça après
                # Increment j to j + 1
                print("{} doesn't exist in twitter dataset".format(w))
            j = j + 1
        '''    
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+1
        '''
    return X_indices

In [38]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_length)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = max_length)

IndexError: index 60 is out of bounds for axis 1 with size 60

In [38]:
type(X_train)

numpy.ndarray

In [ ]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_length = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

### Embedding() layer in Keras
- input:    output of sentences_to_indices()
- output:   word embeddings for a sentence

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False) 

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

### LSTM layers
- input:  array of sentences of shape (m, max_len, ) 
- output: softmax probability vector of shape (m, C = 2)

In [ ]:
def smiley_LSTM(input_shape, word_to_vec_map, word_to_index, dropout_rate):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    
    # Add dropout with a probability of 0.5
    X = Dropout(dropout_rate)(X)
    
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences = False, return_state = False)(X)
    
    # Add dropout with a probability of 0.5
    X = Dropout(dropout_rate)(X)
    
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(5)(X)
    
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [ ]:
def complete_model(word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                   epochs = 50, loss ='categorical_crossentropy', optimizer ='adam'):
    
    model = smiley_LSTM((max_length,), word_to_vec_map, word_to_index, dropout_rate)
    
    if summary:
        model.summary()
        
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    history = model.fit(X_train_indices, Y_train_oh, epochs = epochs, batch_size = batch_size, validation_split = 0.1, shuffle=True)
    
    return history, model

In [ ]:
history, model_lstm = complete_model(word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                                     epochs = 50, loss ='categorical_crossentropy', optimizer ='adam')

In [ ]:
model_lstm.save_weights("model_lstm.h5")

### Analysis of training

In [ ]:
# taken from my miniproject in ANN course
plot_history(history, 'history of model 1')

### Submission

In [ ]:
y_pred = model.predict(X_test_indices)
print("Ready for submission ")

In [ ]:
create_csv_submission(ids, y_pred, 'submissions/submission_model_LSTM_basic.csv')